In [17]:
import pandas as pd
import numpy as np
import  seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,r2_score,mean_absolute_percentage_error,mean_squared_error
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [18]:
df=pd.read_csv("Data/std.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [19]:
X=df.drop(columns=["math score"],axis=1)

In [20]:
y=df["math score"]

In [21]:
num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose  import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [22]:
feature_name=df.columns
feature_name

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [23]:
X = pd.DataFrame(X, columns=feature_name)  

X_transformed = preprocessor.fit_transform(X)
X_transformed.shape


(1000, 19)

In [24]:
X=preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape

((800, 19), (800,))

In [26]:
def evaluate__model(true,predited):
    mae=mean_absolute_error(true,predited)
    mse=mean_squared_error(true,predited)
    rmse=np.sqrt(mean_squared_error(true,predited))
    r2_score=r2_score(true,predited)
    return mae,rmse,r2_score,mse

In [ ]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-NeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor()
}

model_list = []
r2_list =[]

for name, model in models.items():  
   
    model.fit(X_train, y_train)
    
  
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)  
    
   
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(name)
    model_list.append(name)
    r2_list.append(model_test_r2)  
    
    print("Model performance for Training set")
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae)) 
    print("- R2 Score: {:.4f}".format(model_train_r2))
    
    print("--------------------------------")
    
    print("=" * 35)
    print("\n")
    

LinearRegression
Model performance for Training set
- Root Mean Squared Error: 5.3231
- Mean Absolute Error: 4.2667
- R2 Score: 0.8743
--------------------------------


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5938
- Mean Absolute Error: 5.2063
- R2 Score: 0.8071
--------------------------------


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
--------------------------------


K-NeighborsRegressor
Model performance for Training set
- Root Mean Squared Error: 5.7077
- Mean Absolute Error: 4.5167
- R2 Score: 0.8555
--------------------------------


DecisionTreeRegressor
Model performance for Training set
- Root Mean Squared Error: 0.2795
- Mean Absolute Error: 0.0187
- R2 Score: 0.9997
--------------------------------


RandomForestRegressor
Model performance for Training set
- Root Mean Squared Error: 2.3200
- Mean Absolute Error: 1.8432
- R2 Score: 0.9761
-----------------------

In [36]:
ff=pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
ff

,Model Name,R2_Score
2,Ridge,0.880593
0,LinearRegression,0.880433
7,CatBoostRegressor,0.851632
5,RandomForestRegressor,0.851042
8,AdaBoostRegressor,0.850934
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-NeighborsRegressor,0.783813
4,DecisionTreeRegressor,0.732060
